This notebook is structured to read levels from an xml and perform sample calculations. We start off by installing any missing packages this notebook will need to fully run.

In [1]:
import sys, subprocess, importlib.util
required = {'numpy','lvlspy','matplotlib','scipy'}
installed = {pkg for pkg in required if importlib.util.find_spec(pkg) is not None}
missing = required - installed
if missing:
    subprocess.check_call([sys.executable, '-m','pip','install','--quiet',*missing])


import numpy as np
import lvlspy.spcoll as lc
import matplotlib.pyplot as plt

from lvlspy.io import xml
from lvlspy.calculate import evolve

Begin by downloading an example xml file, called, appropriately enough, *example.xml*, from [OSF](https://osf.io/3f59u/).  You may instead place or upload your own file and use it by commenting out the *curl* command.

In [ ]:
!curl -o example.xml -J -L https://osf.io/w6ndg/download

Begin by creating a new species collection.

In [3]:
new_coll = lc.SpColl()

Now ensure that the data in *example.xml* are appropriate to use with *lvlspy* by validating the XML file against the appropriate liblvls schema.

In [4]:
xml.validate('example.xml')

Now read the data into the species collection by updating the (empty) collection with data from the XML file.

In [5]:
xml.update_from_xml(new_coll,'example.xml')

Let's extract the species from the collection and set a temperature (in Kelvin)

In [6]:
sp = new_coll.get()['my_species']
T = 1.e+9 #K

From the stored properties in the xml, energies, Einstein A coefficients, and multiplicities, with the supplied temperature, we can calculate the rate matrix

In [ ]:
rate_matrix = sp.compute_rate_matrix(T)

print('\nRate Matrix:\n')

for i in range(rate_matrix.shape[0]):
    for j in range(rate_matrix.shape[1]):
        print(i, j, rate_matrix[i, j])

Now that we have a rate matrix, we can evolve our species at a fixed temperature. The API contains two methods built into the 'evolve' suite and will be demonstrated. The method to be demonstrated is the Newton-Raphson solver. The solver will output both the abundance and fugacity evolutions of each level.

In [8]:
#here we set the initial conditions for this example 
#we are setting the abundances to be totally in the 
#lowest level
y0 = np.zeros(rate_matrix.shape[0]) 
y0[0] = 1.0

#here we define the time array
time = np.logspace(-30,2,200)

#the method takes in 4 input and outputs 2
#it takes the species and temperature to 
#calculate the rate matrix for it self. It
#also takes the initial condition and the time
#array to be integrated over. The method returns
#the abundances and the fugacities respectively as 2D arrays. They
#have the shape [rate_matrix.shape[0],len(time)]
y,fug = evolve.newton_raphson(sp,T,y0,time)

The API can also compute the equilibrium abundances directly at a given temperature

In [9]:
eq_probs = sp.compute_equilibrium_probabilities(T)

Let's plot the output starting with the abundances and overlay them with the equilibrium abundances

In [ ]:
plt.figure()

for i in range(rate_matrix.shape[0]):
    plt.plot(time,y[i,:], label = str(i) + ' level')

plt.gca().set_prop_cycle(None)  # Reset the color cycle to align equilibria with network solutions.

for i in range(len(eq_probs)):
    plt.plot(time[-1], eq_probs[i], 'x')

plt.xlabel('time (s)')
plt.ylabel('Probability')
plt.xlim([1e-8,1000])
plt.xscale('log')
plt.legend()
plt.show()

As we can see, the solver rightly predicts the equilibrium abundances. The fugacity is defined as the likelyhood the species is to leave a level into the next. Levels with the same fugacity will not exchange abundances, and transitions will happen from levels with higher fugacity into lower ones. Equilibrium is achieved when all the levels have the same fugacity of 1

In [ ]:
plt.figure()
for i in range(fug.shape[0]):
    plt.plot(time,fug[i,:])
plt.xlabel('time (s)')
plt.ylabel('Fugacity')
plt.xscale('log')
plt.xlim([1e-6,1000])
plt.show()

The second method the API contains involes a sparse matrix solver. The method has the same input and output as the Newton-Raphson solver


In [12]:
y, fug = evolve.csc(sp,T,y0,time)

Graphing the solution we get

In [ ]:
plt.figure()

for i in range(rate_matrix.shape[0]):
    plt.plot(time,y[i,:], label = str(i) + ' level')

plt.gca().set_prop_cycle(None)  # Reset the color cycle to align equilibria with network solutions.

for i in range(len(eq_probs)):
    plt.plot(time[-1], eq_probs[i], 'x')

plt.xlabel('time (s)')
plt.ylabel('Probability')
plt.xlim([1e-8,1000])
plt.xscale('log')
plt.legend()
plt.show()

And the fugacity

In [ ]:
plt.figure()
for i in range(fug.shape[0]):
    plt.plot(time,fug[i,:])
plt.xlabel('time (s)')
plt.ylabel('Fugacity')
plt.xscale('log')
plt.xlim([1e-6,1000])
plt.show()

Obviously the results should be the same between both methods. But one would notice the time to execute difference between the two. For such a small system, the Newton-Raphson technique is much faster. For larger systems, the sparse solver will be much faster.

As a final step to illustrate another *xml* method, we output the data in the species collection to new files.  The first uses the default energy scale (*keV*) while the second specifically selects *eV* as the energy scale for the levels.

In [ ]:
xml.write_to_xml(new_coll,'new_example.xml') #writing the original collection to a new xml file

xml.write_to_xml(new_coll,'new_example_ev.xml',units = 'eV') #writes the same collection to xml but converts all the energies to eV

!cat new_example_ev.xml